In [1]:
import pandas as pd
import time

In [2]:
import math

def policy(params, substep, state_history, previous_state):
    return {'step_size': 1}

def update_a(params, substep, state_history, previous_state, policy_input):
    a = b = c = d = e = 100.0
    return 'a', previous_state['a'] * abs(math.cos(previous_state['a']))

def update_b(params, substep, state_history, previous_state, policy_input):
    return 'b', previous_state['b'] + policy_input['step_size'] * params['a']

params = {
    'a': [1, 2],
    'b': [1]
}

states = {
    'a': 1.0,
    'b': 2.0
}

psubs = [
    {
        'policies': {},
        'variables': {
            'a': update_a
        }
    },
    {
        'policies': {
            'p_1': policy,
            'p_2': policy,
            'p_3': policy,
            'p_4': policy,
            'p_5': policy,
        },
        'variables': {
            'b': update_b
        }
    }
]

---

In [3]:
TIMESTEPS = 100_000
RUNS = 1

In [4]:
import output.rad_cad as rc
from output.rad_cad import Model, Simulation

model = Model(initial_state=states, psubs=psubs, params=params)
simulation = Simulation(model=model, timesteps=TIMESTEPS, runs=RUNS)

average_duration = 0
for i in range(10):
    start = time.time()
    data_rc = rc.run([simulation])
    end = time.time()
    duration = end - start
    average_duration = (duration + average_duration) / 2
    print(average_duration)

2.2233335971832275
3.049302339553833
3.5730446577072144
3.9381527304649353
4.025483936071396
3.8684355467557907
4.1192512139678
4.071369033306837
4.094631126150489
4.216228093020618


In [ ]:
df_radcad = pd.DataFrame(data_rc)
print(df_radcad)

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor

RUNS_THREADED = 2

executor = ThreadPoolExecutor(max_workers=RUNS_THREADED)

t0 = time.time()

futures = []
results = []
for run in range(RUNS_THREADED):
    future = executor.submit(rc.run, TIMESTEPS, 1, states, psubs, params)
    futures.append(future)
    result = future.result()
    results.append(result)
t1 = time.time()

print(f"Time: {t1 - t0}s")

---

In [5]:
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionMode, ExecutionContext
from cadCAD.engine import Executor

In [6]:
from cadCAD import configs
del configs[:] # Clear any prior configs

config = {
    "N": RUNS,
    "T": range(TIMESTEPS),
    "M": params
}

c = config_sim(config)

exp = Experiment()
exp.append_configs(
    initial_state = states,
    partial_state_update_blocks = psubs,
    sim_configs = c
)

In [7]:
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(context=exec_mode.multi_mode)

In [8]:
simulation = Executor(exec_context=local_mode_ctx, configs=configs)

In [9]:
start = time.time()
data, tensor_field, sessions = simulation.execute()
end = time.time()
print(end - start)

df_cadcad = pd.DataFrame(data)
print(df_cadcad)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: multi_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100000, 2, 2, 7)
Execution Method: parallelize_simulations
Execution Mode: parallelized
Total execution time: 10.50s
10.511757135391235
               a          b  simulation  subset  run  substep  timestep
0       1.000000        2.0           0       0    1        0         0
1       0.540302        2.0           0       0    1        1         1
2       0.540302        7.0           0       0    1        2         1
3       0.463338        7.0           0       0    1        1         2
4       0.463338       12.0           0       0    1        2         2
...          ...        ...         ...     ...  ...      ...       ...
399997  0.003162   999982.0         

In [ ]:
from pandas.testing import assert_frame_equal

assert_frame_equal(df_radcad.drop(['run'], axis=1), df_cadcad.drop(['run'], axis=1))

---

In [ ]:
time_profile = []

for i in range(7):
    timesteps = 10**i
    start = time.time()
    data_rc = rc.run(timesteps, 1, states, psubs, params)
    end = time.time()
    duration = end - start
    time_profile.append({'timesteps': timesteps, 'duration': duration})

In [ ]:
pd.DataFrame(time_profile).plot(x='timesteps', y='duration')

---

In [ ]:
%%capture
time_profile_cadcad = []

for i in range(7):
    timesteps = 10**i
    
    from cadCAD import configs
    del configs[:] # Clear any prior configs

    config = {
        "N": 1,
        "T": range(timesteps),
        "M": params
    }

    c = config_sim(config)

    exp = Experiment()
    exp.append_configs(
        initial_state = states,
        partial_state_update_blocks = psubs,
        sim_configs = c
    )
    
    exec_mode = ExecutionMode()
    local_mode_ctx = ExecutionContext(context=exec_mode.local_mode)
    
    simulation = Executor(exec_context=local_mode_ctx, configs=configs)
    
    start = time.time()

    data, tensor_field, sessions = simulation.execute()

    end = time.time()
    duration = end - start
    time_profile_cadcad.append({'timesteps': timesteps, 'duration': duration})

In [ ]:
pd.DataFrame(time_profile_cadcad).plot(x='timesteps', y='duration')

---

In [ ]:
df_rc = pd.DataFrame(time_profile)
df_cc = pd.DataFrame(time_profile_cadcad)

ax = df_rc.plot(x='timesteps', y='duration')
df_cc.plot(ax=ax, x='timesteps', y='duration')

In [ ]:
time_profile_cadcad[6]['duration']/time_profile[6]['duration']

---

In [ ]:
result = [states]

def run():
    for i in range(TIMESTEPS):
        previous_state = result[-1]
        next_state = {}
        for psub in psubs:
            for state in psub['updates']:
                next_state.update({
                    state: psub['updates'][state](previous_state)
                })
        result.append(next_state)
    return result

start = time.time()
data = run()
end = time.time()
print(end - start)

df = pd.DataFrame(data)
df